In [3]:
from data_algebra.data_ops import *  # https://github.com/WinVector/data_algebra

import data_algebra.MySQL


d = data_algebra.default_data_model.pd.DataFrame({
    'x': [1, 2, 3],
    'g': ['a', 'a', 'b'],
})


ops = (
    describe_table(d, table_name='d')
        .extend({'sx': 'x.sum()'})
        .extend({'og1': '(1).cumsum()'},
                partition_by=['g'],
                order_by=['x'])
        .extend({'og2': '(1).cumsum()'},
                partition_by=['g'],
                order_by=['x'], reverse=['x'])
        .extend({'og1': '(5).cumsum()'},
            order_by=['x'], reverse=['x'])
)

print(ops)

TableDescription(
 table_name='d',
 column_names=[
   'x', 'g']) .\
   extend({
    'sx': 'x.sum()'},
   partition_by=1) .\
   extend({
    'og1': '(1).cumsum()'},
   partition_by=['g'],
   order_by=['x']) .\
   extend({
    'og2': '(1).cumsum()'},
   partition_by=['g'],
   order_by=['x'],
   reverse=['x']) .\
   extend({
    'og1': '(5).cumsum()'},
   partition_by=1,
   order_by=['x'],
   reverse=['x'])


In [4]:
# show the SQL is shortened
db_handle = data_algebra.MySQL.MySQLModel().db_handle(conn=None)
sql = db_handle.to_sql(ops, pretty=True, use_with=False, annotate=True)

print(sql)

-- data_algebra SQL https://github.com/WinVector/data_algebra
--  dialect: MySQLModel
--       string quote: '
--   identifier quote: `
SELECT -- extend({ 'sx': 'x.sum()'}, partition_by=1).extend({ 'og2': '(1).cumsum()'}, partition_by=['g'], order_by=['x'], reverse=['x']).extend({ 'og1': '(5).cumsum()'}, partition_by=1, order_by=['x'], reverse=['x'])
 `x`,
 `g`,
 SUM(`x`) OVER () AS `sx`,
               SUM(1) OVER (PARTITION BY `g`
                            ORDER BY `x` DESC) AS `og2`,
                           SUM(5) OVER (
                                        ORDER BY `x` DESC) AS `og1`
FROM `d`
